1 take the y plates, and some data preprocessing below:

In [ ]:
 import pandas as pd

# Load the original data
file_path = 'applications.csv'
data = pd.read_csv(file_path)

# Filter rows where the 'status' column contains 'Y'
data_filtered_y = data[data['status'] == 'Y']

# Replace NaN values with empty strings for easier processing
data_filtered_y['customer_meaning'] = data_filtered_y['customer_meaning'].fillna('')

# Define a function to clean up the meaning
def clean_meaning(row):
    customer_meaning = row['customer_meaning'].lower()
    if 'quickweb' in customer_meaning or 'micro' in customer_meaning or customer_meaning == '':
        return row['reviewer_comments']  # Take the reviewer's meaning
    else:
        return row['customer_meaning']   # Keep the original meaning if valid

# Apply the cleaning function to create a new 'meaning' column
data_filtered_y['customer_meaning'] = data_filtered_y.apply(clean_meaning, axis=1)

# Select only the necessary columns
final_data = data_filtered_y[['plate', 'customer_meaning']]

# Save the cleaned data to a new CSV file
output_file_path = 'filtered_plates.csv'
final_data.to_csv(output_file_path, index=False)

# Display the path of the new CSV file
print(output_file_path)


2. classify using AgglomerativeClustering and plot

In [ ]:
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.decomposition import PCA

# Load the filtered data
filtered_data = pd.read_csv('filtered_plates.csv')

# Convert all customer_meaning entries to strings and remove NaN values
filtered_data['customer_meaning'] = filtered_data['customer_meaning'].astype(str).fillna('')
valid_meanings = filtered_data[filtered_data['customer_meaning'] != '']

# Load the SentenceTransformer model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight and effective

# Generate embeddings for the "meaning" column
print("Generating embeddings for meanings...")
embeddings = embedding_model.encode(valid_meanings['customer_meaning'].tolist())

# Tune number of clusters using silhouette score
range_n_clusters = range(2, 11)
silhouette_scores = []
'''
for n_clusters in range_n_clusters:
    print(f"Tuning number of clusters: {n_clusters}")
    agglo_clf = AgglomerativeClustering(n_clusters=n_clusters)
    labels = agglo_clf.fit_predict(embeddings)
    score = silhouette_score(embeddings, labels)
    silhouette_scores.append(score)


# Plot silhouette scores
plt.figure(figsize=(10, 6))
plt.plot(range_n_clusters, silhouette_scores, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs Number of Clusters for Agglomerative Clustering')
plt.show()

# Find the best number of clusters
best_n_clusters = range_n_clusters[np.argmax(silhouette_scores)]
print(f"Best number of clusters: {best_n_clusters}")


# Fit Agglomerative Clustering with the best number of clusters
'''
best_n_clusters = 8
agglo_clf = AgglomerativeClustering(n_clusters=best_n_clusters)
labels = agglo_clf.fit_predict(embeddings)

# Add labels to the dataframe
valid_meanings['cluster_label'] = labels

# Print the number of data points in each cluster
cluster_counts = valid_meanings['cluster_label'].value_counts()
print("\nNumber of data points in each cluster:")
print(cluster_counts)

# Visualize clusters using PCA to reduce dimensions to 2D
pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(embeddings)

plt.figure(figsize=(10, 6))
for cluster in range(best_n_clusters):
    cluster_points = reduced_embeddings[labels == cluster]
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], label=f'Cluster {cluster}', s=10)  # Adjust 's' to make dots smaller


plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('Clusters Visualization after PCA')
plt.legend()
plt.show()

# Save the resulting dataframe to CSV with cluster labels 0-4 for each data point
valid_meanings.to_csv('clustered_plates_with_labels.csv', index=False)

print("Output saved to 'clustered_plates_with_labels.csv'")


3 sample certain amount of points from each clucter and

 manually copy to gpt to get the meaning for each cluster(you dont need to do)

In [ ]:
import pandas as pd

# Load the clustered plates with labels data
file_path = 'clustered_plates_with_labels.csv'  # Ensure the file is in the correct directory
clustered_data = pd.read_csv(file_path)

# Loop through each unique cluster label and sample 50 data points
for cluster_label in clustered_data['cluster_label'].unique():
    sampled_data = clustered_data[clustered_data['cluster_label'] == cluster_label].sample(n=30, replace=True)
    print(f"\nCluster {cluster_label} has samples:")
    print(sampled_data[['plate', 'customer_meaning']].to_string(index=False))



4 assignment meanings based on cluster numbers, note that different cluster number may corrsponding different cluster meaning for different run like my cod 0 represent "Personal and Family names", but when you run, 0 may represent different meaning, so you many manually check from previous output to see which meaning crrsponding to which number.

In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import pandas as pd

# Load the clustered plates with labels data
file_path = 'john_clustered_plates_part2.csv'
clustered_data = pd.read_csv(file_path)

# Define a dictionary that maps cluster labels to the corresponding meaning
cluster_meanings = {
    0: "Personal and Family names",
    1: "Business and University Identities",
    2: "Vehicle and Car Descriptions and Attributes",
    3: "Diverse Personal Expressions",
    4: "Area code and Numbers",
    5: "Gang Color and Controversial Affiliations",
    6: "Pet and Animal Themes",
    7: "Colors and Descriptive Traits"
}

# Add a new column with the corresponding cluster meaning
clustered_data['cluster_meaning'] = clustered_data['cluster_label'].map(cluster_meanings)

# Save the updated dataframe to a new CSV file
output_file_path = 'clustered_plates_with_label_meanings.csv'
clustered_data.to_csv(output_file_path, index=False)

# Display the path of the new CSV file
print(output_file_path)


clustered_plates_with_label_meanings.csv


5:0 shot using three other different llm, which takes long time

In [ ]:
import pandas as pd
from transformers import pipeline
import csv


# Load the clustered plates with labels data
file_path = 'clustered_plates_with_label_meanings.csv'
clustered_data = pd.read_csv(file_path)

# Ensure all customer_meaning are strings and handle NaNs
clustered_data['customer_meaning'] = clustered_data['customer_meaning'].fillna('missing').astype(str)

# Define the cluster meanings dictionary
cluster_meanings = {
    0: "Personal and Family names",
    1: "Business and University Identities",
    2: "Vehicle and Car Descriptions and Attributes",
    3: "Diverse Personal Expressions",
    4: "Area code and Numbers",
    5: "Gang Color and Controversial Affiliations",
    6: "Pet and Animal Themes",
    7: "Colors and Descriptive Traits"
}

cluster_meanings_count = {
    "Personal and Family names": 0,
    "Business and University Identities": 0,
    "Vehicle and Car Descriptions and Attributes": 0,
    "Diverse Personal Expressions": 0,
    "Area code and Numbers": 0,
    "Gang Color and Controversial Affiliations": 0,
    "Pet and Animal Themes": 0,
    "Colors and Descriptive Traits": 0
}

# Initialize three different zero-shot classifiers
classifier_1 = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device="cuda")
classifier_2 = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-3", device="cuda")
classifier_4 = pipeline("zero-shot-classification", model="joeddav/xlm-roberta-large-xnli", device="cuda")

# Open a CSV file to write the results
output_file_path = 'john_clustered_plates_with_llm_classifications.csv'
with open(output_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(['plate', 'customer_meaning', 'original_class', 'facebook_class', 'valhalla_class', 'xlm_class', 'majority_vote'])

    # Iteratively classify each data point using three LLMs
    for index, row in clustered_data.iterrows():
        plate = row['plate']
        customer_meaning = row['customer_meaning']
        original_class = row['cluster_meaning']

        # Classify using LLM 1
        result_1 = classifier_1(customer_meaning, list(cluster_meanings.values()))
        llm1_class = result_1['labels'][0]

        # Classify using LLM 2
        result_2 = classifier_2(customer_meaning, list(cluster_meanings.values()))
        llm2_class = result_2['labels'][0]

        # Classify using LLM 4
        result_4 = classifier_4(customer_meaning, list(cluster_meanings.values()))
        llm4_class = result_4['labels'][0]
        cluster_meanings_count_copy = cluster_meanings_count.copy()

        cluster_meanings_count_copy[original_class] += 1
        cluster_meanings_count_copy[llm1_class] += 1
        cluster_meanings_count_copy[llm2_class] += 1
        cluster_meanings_count_copy[llm4_class] += 1

        max_meaning_count = 0
        for key, val in cluster_meanings_count_copy.items():
            max_meaning_count = max(max_meaning_count, val)
        max_count = 0
        majority_vote = None
        for key, val in cluster_meanings_count_copy.items():
            if val == max_meaning_count:
                max_count += 1
                majority_vote = key
        if max_count > 1:
            majority_vote = None

        # Write the row to the CSV file
        writer.writerow([plate, customer_meaning, original_class, llm1_class, llm2_class, llm4_class, majority_vote])

        # Enhanced print statement to accurately reflect progress
        print(f"Processed {index + 1}/{len(clustered_data)}: Plate = {plate}, Meaning = {customer_meaning}, Original Class = {original_class}, LLM1 Class = {llm1_class}, LLM2 Class = {llm2_class}, LLM4 Class = {llm4_class}, Majority Vote = {majority_vote}")


Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Processed 1/1557: Plate = MTY WHTY, Meaning = IT’S A WHITE 2015 MINI COOPER S "MIGHTY WHITEY", Original Class = Vehicle and Car Descriptions and Attributes, LLM1 Class = Vehicle and Car Descriptions and Attributes, LLM2 Class = Vehicle and Car Descriptions and Attributes, LLM4 Class = Diverse Personal Expressions, Majority Vote = Vehicle and Car Descriptions and Attributes
Processed 2/1557: Plate = 4RMCALI, Meaning = IM FROM CA, Original Class = Business and University Identities, LLM1 Class = Area code and Numbers, LLM2 Class = Area code and Numbers, LLM4 Class = Diverse Personal Expressions, Majority Vote = Area code and Numbers
Processed 3/1557: Plate = DAGOOZE, Meaning = THE GOOSE- A NICKNAME FOR MOTO GUZZI M/C, Original Class = Diverse Personal Expressions, LLM1 Class = Pet and Animal Themes, LLM2 Class = Vehicle and Car Descriptions and Attributes, LLM4 Class = Diverse Personal Expressions, Majority Vote = Diverse Personal Expressions
Processed 4/1557: Plate = BRZN BLU, Meaning =

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Processed 10/1557: Plate = 2J2O9Y6, Meaning = PERSONAL, Original Class = Diverse Personal Expressions, LLM1 Class = Diverse Personal Expressions, LLM2 Class = Diverse Personal Expressions, LLM4 Class = Diverse Personal Expressions, Majority Vote = Diverse Personal Expressions
Processed 11/1557: Plate = G$JCK 3, Meaning = G AS IN GANGSTER, Original Class = Gang Color and Controversial Affiliations, LLM1 Class = Gang Color and Controversial Affiliations, LLM2 Class = Gang Color and Controversial Affiliations, LLM4 Class = Diverse Personal Expressions, Majority Vote = Gang Color and Controversial Affiliations
Processed 12/1557: Plate = BLU TII, Meaning = BLUE, Original Class = Colors and Descriptive Traits, LLM1 Class = Diverse Personal Expressions, LLM2 Class = Gang Color and Controversial Affiliations, LLM4 Class = Colors and Descriptive Traits, Majority Vote = Colors and Descriptive Traits
Processed 13/1557: Plate = BLUTONC, Meaning = THE NAME I USE FOR ONLINE GAMING, Original Class = 

ModuleNotFoundError: No module named 'google.colab'

In [11]:
!pip install sentencepiece

6 write the code or suing gpt to write the codeto vote for meaning and take the majority one, and manually fill blank